In [7]:
import numpy as np
import collections
import pandas as pd
import itertools
import seaborn as sns
import time
import json
import re
import os

print os.getcwd()

/Users/asd/coding/ingredient2vec


In [27]:
'''
Load data & Path

'''

path = 'data/'
ingr_info = 'ingr_info.tsv'
comp_info = 'comp_info.tsv'
ingr_comp = 'ingr_comp.tsv'

def load_ingredients(path):
    ingredients = {}
    ingredients_list = []
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                pass
            else:
                line_split = line.rstrip().split('\t')
                ingredients_id = line_split[0]
                ingredients_list = line_split[1:]
                ingredients[ingredients_id] = ingredients_list
    return ingredients

def load_compounds(path):
    compounds = {}
    compounds_list = []
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                pass
            else:
                line_split = line.rstrip().split('\t')
                compounds_id = line_split[0]
                compounds_list = line_split[1:]
                compounds[compounds_id] = compounds_list
    return compounds

def load_relations(path):
    relations = {}
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                pass
            else:
                line_split = line.rstrip().split('\t')
                ingredients_id = line_split[0]
                compounds_id = line_split[1]
                
                if ingredients_id in relations:
                    relations[ingredients_id].append(compounds_id)
                    
                else:
                    relations[ingredients_id] = [compounds_id]
                    
    return relations



ingredients = load_ingredients(path+ingr_info)
compounds = load_compounds(path+comp_info)
relations = load_relations(path+ingr_comp)

In [56]:
'''
Relation Analysis
Ingredients & Compounds
'''

print '# of ingredients:', len(ingredients)
print '# of compounds:', len(compounds)

print '\n############################\n'

print '# of ingredients with relations:', len(relations)

for rel in relations:
    ingr, comps = rel[0], rel[1]
    if len(comps) == 20:
        print ingredients[ingr][0]

# of ingredients: 1530
# of compounds: 1107

############################

# of ingredients with relations: 1525
buchu_oil
marjoram
toasted_sesame_seed
roasted_macadamia_nut
eucalyptus
